In [1]:
#import packages
import pandas as pd
import numpy as np

In [9]:
#get the expression data, we use ko dataset first
ko_df = pd.read_csv("C:/Users/yutin/Desktop/CS598SS_GRN-master/dataset/expression/KO.txt", sep = '\t')
nv_df = pd.read_csv('C:/Users/yutin/Desktop/CS598SS_GRN-master/dataset/expression/NatVar.txt', sep='\t')
stress_df = pd.read_csv('C:/Users/yutin/Desktop/CS598SS_GRN-master/dataset/expression/Stress.txt', sep='\t')

#get the TF data
ko_tf = pd.read_csv("C:/Users/yutin/Desktop/CS598SS_GRN-master/dataset/expression/KO_TF_names.txt", sep = '\t', header= None)
ko_tf.columns = ['TF']

nv_tf = pd.read_csv('C:/Users/yutin/Desktop/CS598SS_GRN-master/dataset/expression//NatVar_TF_names.txt', sep='\t', header=None)
nv_tf.columns = ['TF']

stress_tf = pd.read_csv('C:/Users/yutin/Desktop/CS598SS_GRN-master/dataset/expression/Stress_TF_names.txt', sep='\t', header=None)
stress_tf.columns = ['TF']

In [15]:
#seprate the tfs from dfs
#use ko dataset as our first attempt here
new_ko_df = ko_df[~ko_df['Gene'].isin(ko_tf['TF'])]
new_ko_df = new_ko_df.set_index('Gene')

new_ko_tf = ko_df[ko_df['Gene'].isin(ko_tf['TF'])]
new_ko_tf = new_ko_tf.set_index('Gene')


In [23]:
#prepare dataset, set target gene(Y) and predictors(X)
from sklearn import model_selection
def prep_dataset(new_tf, new_df):
    '''
    Args:
        -new_tf, the predictors(X), already in dataframe format
        -new_df, the taget genes(Y), already in dataframe format
    Returns:
        -training and testing sets
        -label for predictors
    '''
    y = new_df.values
    X = new_tf

    X_label = X.index #gene names for predictors
    X = X.values.transpose() # why transpose????

    #75:25 split
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.25)

    return X_train, X_test, y_train, y_test, X_label



    

Lasso regression

In [17]:

from sklearn.linear_model import Lasso, LassoCV

def grn_lasso(new_tf, new_df):
    '''
    GRN construction using lasso regression
    Args:
        -new_tf, already in dataframe format
        -new_df, already in dataframe format
    return:
        -non-zero predictors
    '''
    #prep data
    X_train, X_test, y_train, y_test, X_label = prep_dataset(new_tf, new_df)

    #use lasso regression
    lasso_reg = LassoCV(alphas=[0.01, 0.1, 1, 10, 100], cv = 10)
    lasso_reg.fit(X_train, y_train)

    #get the scores
    train_score = lasso_reg.score(X_train, y_train)
    test_score = lasso_reg.score(X_test, y_test)

    #find the non-zero predictors as regulators
    nonzero_filter = lasso_reg.coef_ != 0
    regulators = X_label[nonzero_filter]

    return regulators.values

Ridge regression

In [19]:
from sklearn.linear_model import Ridge, RidgeCV

def grn_ridge(new_tf, new_df):
    '''
    GRN construction using ridge regression, I guess it is going to show similar outcome as lasso
    Args:
        -new_tf, already in dataframe format
        -new_df, already in dataframe format
    return:
        -non-zero predictors
    '''
    #prep data
    X_train, X_test, y_train, y_test, X_label = prep_dataset(new_tf, new_df)

    #use lasso regression
    ridge_reg = RidgeCV(alphas=[0.01, 0.1, 1, 10, 100], cv = 10)
    ridge_reg.fit(X_train, y_train)

    #get the scores
    train_score = ridge_reg.score(X_train, y_train)
    test_score = ridge_reg.score(X_test, y_test)

    #find the non-zero predictors as regulators
    nonzero_filter = ridge_reg.coef_ != 0
    regulators = X_label[nonzero_filter]

    return regulators.values

In [24]:
#first try
grn_lasso(new_ko_tf, new_ko_df)

ValueError: Found input variables with inconsistent numbers of samples: [536, 5441]